<a href="https://colab.research.google.com/github/saparbayev-azizbek-12/bi-and-ai-talents-dl/blob/main/lesson-23/lesson_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import torch
import torch.nn as nn
from datasets import Dataset
from google.colab import drive
from transformers import DataCollatorForSeq2Seq
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# config
USER_TOKEN = '<|user|>'
END_TOKEN = '<|endoftext|>'
ASSISTANT_TOKEN = '<|assistant|>'
MODEL_NAME = "microsoft/Phi-3.5-mini-instruct"

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [ ]:
# dataset
drive.mount('/content/drive')
base_dir = '/content/drive/MyDrive/dl_course/lesson-22/'
file_name = 'instruction-data.json'
data = json.load(open(base_dir + file_name))
raw_ds = Dataset.from_list(data)

Mounted at /content/drive


In [ ]:
# model & tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map=device)

config.json: 0.00B [00:00, ?B/s]

This model config has set a `rope_parameters['original_max_position_embeddings']` field, to be used together with `max_position_embeddings` to determine a scaling factor. Please set the `factor` field of `rope_parameters`with this ratio instead -- we recommend the use of this field over `original_max_position_embeddings`, as it is compatible with most model architectures.


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/195 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [ ]:
def apply_chat_template(example):
    user_text = example['instruction'] + '\n' + example['input']
    ai_text = example['output']
    tokenized = tokenizer.apply_chat_template([
        {'role': 'user', 'content': user_text},
        {'role': 'assistant', 'content': ai_text}
    ], tokenize=True)

    assistant_token_id = tokenizer(ASSISTANT_TOKEN)
    end_token_id = tokenizer(END_TOKEN)

    start = tokenized['input_ids'].index(assistant_token_id['input_ids'][0]) + 1
    end = tokenized['input_ids'].index(end_token_id['input_ids'][0]) + 1

    labels = [-100] * len(tokenized['input_ids'])
    for i in range(start, end):
      labels[i] = tokenized['input_ids'][i]


    return {
        'input_ids': torch.tensor(tokenized['input_ids']),
        'attention_mask': torch.tensor(tokenized['attention_mask']),
        'labels': torch.tensor(labels)
    }

In [ ]:
train_ds = raw_ds.map(apply_chat_template, remove_columns=raw_ds.column_names)

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir=f'{MODEL_NAME}-finetuned',
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    optim='adamw_torch',
    report_to='none'
)
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model
)
trainer = Trainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_ds,
    data_collator=data_collator
)

In [ ]:
model.lm_head = nn.Sequential(
    nn.Linear(3072, 4 * 3072),
    nn.Linear(4 * 3072, 4 * 3072),
    nn.Linear(4 * 3072, 32064)
)

In [ ]:
model

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLUActivation()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
    (rotary_emb): Phi3RotaryEmbedding()
  )
  (lm_head): Sequential(
    (0): Linear(in_feat

In [ ]:
for param in model.parameters():
  param.require_grad = False

In [ ]:
for param in model.lm_head.parameters():
  param.require_grad = True

In [ ]:
model.lm_head.to(torch.bfloat16)
trainer.train()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (424x3072 and 32064x12288)

In [ ]:
import nbformat

In [ ]:

nb = nbformat.read("lesson-23.ipynb", as_version=4)
nb.metadata.pop("widgets", None)
nbformat.write(nb, "your_notebook_clean.ipynb")